In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torch.ao.quantization import (
  get_default_qconfig_mapping,
  get_default_qat_qconfig_mapping,
  QConfigMapping,
)
import torch.ao.quantization.quantize_fx as quantize_fx
from torch.ao.quantization.quantize_fx import prepare_qat_fx, convert_fx
import copy

In [2]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(8, 8*8, 3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10),
            nn.Softmax()
        )

    def forward(self, x):
        y = self.model(x)
        return y
    

class ConvBNClassifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, stride=2)
        self.bn1 = nn.BatchNorm2d(8)
        self.lr1 = nn.LeakyReLU(0.1)
        self.conv2 = nn.Conv2d(8, 8*8, 3, stride=2)
        self.bn2 = nn.BatchNorm2d(8*8)
        self.lr2 = nn.LeakyReLU(0.1)
        
        self.head = nn.Sequential(
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.lr1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.lr2(x)
        y = self.head(x)
        return y

class ConvBN(torch.nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.module = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=2),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.1)
        )
    
    def forward(self, x):
        return self.module(x)
    

class Head(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.module = nn.Sequential(
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10),
            nn.Softmax()
        )

    def forward(self, x):
        return self.module(x)

class ComposedClassifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.convbn1 = ConvBN(1, 8)
        self.convbn2 = ConvBN(8, 8*8)
        self.head = Head()

    def forward(self, x):
        x = self.convbn1(x)
        x = self.convbn2(x)
        y = self.head(x)
        return y


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = ComposedClassifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [4]:
model.to("cuda")
model_to_quantize = copy.deepcopy(model)
qconfig_mapping = get_default_qat_qconfig_mapping("x86")
example_inputs = torch.rand(size=(1,3,28,28)).to("cpu")
prepared_model = prepare_qat_fx(model_to_quantize, qconfig_mapping, example_inputs)

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:220: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [5]:
print(model)
print(prepared_model)
model.eval()
prepared_model.eval()

ComposedClassifier(
  (convbn1): ConvBN(
    (module): Sequential(
      (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2))
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
  )
  (convbn2): ConvBN(
    (module): Sequential(
      (0): Conv2d(8, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
  )
  (head): Head(
    (module): Sequential(
      (0): AvgPool2d(kernel_size=3, stride=2, padding=0)
      (1): Flatten(start_dim=1, end_dim=-1)
      (2): Dropout(p=0.5, inplace=False)
      (3): Linear(in_features=256, out_features=128, bias=True)
      (4): Dropout(p=0.5, inplace=False)
      (5): Linear(in_features=128, out_features=10, bias=True)
      (6): Softmax(dim=None)
    )
  )
)
GraphModule(
  (activation_post_process_0): FusedMovingAvgObsFakeQuantize(
    fake_quant_e

GraphModule(
  (activation_post_process_0): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1], device='cuda:0'), observer_enabled=tensor([1], device='cuda:0'), scale=tensor([1.], device='cuda:0'), zero_point=tensor([0], device='cuda:0', dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
    (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
  )
  (convbn1): Module(
    (module): Module(
      (0): ConvBn2d(
        1, 8, kernel_size=(3, 3), stride=(2, 2)
        (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
          fake_quant_enabled=tensor([1], device='cuda:0'), observer_enabled=tensor([1], device='cuda:0'), scale=tensor([1.], device='cuda:0'), zero_point=tensor([0], device='cuda:0', dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_cha

In [6]:
prepared_model.to("cuda")
optimizer = torch.optim.SGD(prepared_model.parameters(), lr=1e-3)

for i in range(20):
    train(train_dataloader, prepared_model, loss_fn, optimizer)
prepared_model.to("cpu")
quantized_model = convert_fx(prepared_model)
quantized_model.eval()

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/fake_quantize.py:353: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/ReduceAllOps.cpp:71.)
  return torch.fused_moving_avg_obs_fake_quant(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/fake_quantize.py:353: UserWarning: _aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:687.)
  return torch.fused_moving_avg_obs_fake_quant(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*arg

GraphModule(
  (convbn1): Module(
    (module): Module(
      (0): QuantizedConv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), scale=0.06766307353973389, zero_point=67)
      (2): QuantizedLeakyReLU(negative_slope=0.1)
    )
  )
  (convbn2): Module(
    (module): Module(
      (0): QuantizedConv2d(8, 64, kernel_size=(3, 3), stride=(2, 2), scale=0.08459077030420303, zero_point=61)
      (2): QuantizedLeakyReLU(negative_slope=0.1)
    )
  )
  (head): Module(
    (module): Module(
      (0): AvgPool2d(kernel_size=3, stride=2, padding=0)
      (1): Flatten(start_dim=1, end_dim=-1)
      (2): QuantizedDropout(p=0.5, inplace=False)
      (3): QuantizedLinear(in_features=256, out_features=128, scale=0.04332021251320839, zero_point=66, qscheme=torch.per_channel_affine)
      (4): QuantizedDropout(p=0.5, inplace=False)
      (5): QuantizedLinear(in_features=128, out_features=10, scale=0.12145350873470306, zero_point=67, qscheme=torch.per_channel_affine)
      (6): QuantizedSoftmax(dim=None)
    )


In [7]:
%%timeit
test(test_dataloader, prepared_model, loss_fn)

Test Error: 
 Accuracy: 48.5%, Avg loss: 2.031409 

Test Error: 
 Accuracy: 48.5%, Avg loss: 2.031255 

Test Error: 
 Accuracy: 48.6%, Avg loss: 2.031166 

Test Error: 
 Accuracy: 48.6%, Avg loss: 2.032034 

Test Error: 
 Accuracy: 48.5%, Avg loss: 2.031788 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031757 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031686 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031699 

1.22 s ± 93.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
prepared_model

GraphModule(
  (activation_post_process_0): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0079]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
    (activation_post_process): MovingAverageMinMaxObserver(min_val=0.0, max_val=1.0)
  )
  (convbn1): Module(
    (module): Module(
      (0): QuantizedConv2d(Reference)(1, 8, kernel_size=(3, 3), stride=(2, 2))
      (2): LeakyReLU(negative_slope=0.1)
    )
  )
  (activation_post_process_1): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([0.0679]), zero_point=tensor([67], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
    (activation_post_process): MovingAverageMinMaxObserver(min_val=-4.56900691986084, max_val=4.048405647277832)
  )
  (activation_post_

In [9]:
%%timeit
test(test_dataloader, prepared_model, loss_fn)

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031695 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031695 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031699 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031694 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031694 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031694 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031694 

Test Error: 
 Accuracy: 48.4%, Avg loss: 2.031694 

1.25 s ± 3.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
test(test_dataloader, quantized_model, loss_fn)

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/nn/modules/module.py:1541: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return forward_call(*args, **kwargs)


Test Error: 
 Accuracy: 48.7%, Avg loss: 2.031487 

Test Error: 
 Accuracy: 48.7%, Avg loss: 2.031487 

Test Error: 
 Accuracy: 48.7%, Avg loss: 2.031487 

Test Error: 
 Accuracy: 48.7%, Avg loss: 2.031487 

Test Error: 
 Accuracy: 48.7%, Avg loss: 2.031487 

Test Error: 
 Accuracy: 48.7%, Avg loss: 2.031487 

Test Error: 
 Accuracy: 48.7%, Avg loss: 2.031487 

Test Error: 
 Accuracy: 48.7%, Avg loss: 2.031487 

867 ms ± 223 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
torch.save(quantized_model, "model_int8_fx_qat.pt")